# Collect Labels with Simple CNN Model

# Imports

In [ ]:
import sys
sys.path.append("../src")

import os
import torch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm

import re
from io import StringIO
from typing import List, Type
from pathlib import Path
import json
import shutil

from torchvision import transforms
import torch.nn as nn

from cnn import SimpleCNN

# 1. Verify data completeness

In [ ]:
mapping_path = '../data/maps/mapping.csv'
cwd = Path.cwd().parent
video_dir = os.path.join(cwd, 'data', 'videos')

In [ ]:
def check_directories(mapping_path, video_dir_path):
    # Read csv
    df = pd.read_csv(mapping_path)
    # Get count ofrows in csv
    num_rows = len(df)
    print(f"How many rows in csv file: {num_rows}")
    
    # Get count of directories in video folder
    folder_names = [name for name in os.listdir(video_dir_path) if os.path.isdir(os.path.join(video_dir_path, name))]
    num_folders = len(folder_names)
    print(f"How many directories in {video_dir_path}: {num_folders}")
    
    # List missing directories
    expected_folders = set(df['directory'])
    actual_folders = set(folder_names)
    missing_folders = expected_folders - actual_folders
    
    if missing_folders:
        print(f"Missing directories : {missing_folders}")
    else:
        print("All directories are present.")

    return missing_folders

In [ ]:
missing_dir = check_directories(mapping_path, video_dir)

In [ ]:
def copy_missing_directories(missing_folders, destination_path, source_path = '/home/rob/Documents/data/selected_frames'):
    copied_folders = 0
    not_copied_folders = []
    
    for folder in missing_folders:
        src_folder = os.path.join(source_path, folder)
        dest_folder = os.path.join(destination_path, folder)
        
        if os.path.exists(src_folder):
            shutil.copytree(src_folder, dest_folder)
            copied_folders += 1
        else:
            not_copied_folders.append(folder)
    
    print(f"Number of folders copied: {copied_folders}")
    if not_copied_folders:
        print(f"Folders that could not be copied: {not_copied_folders}")


In [ ]:
if missing_dir:
    copy_missing_directories(missing_dir, destination_path=video_dir)

# 2. Inference

## Get Weights

In [ ]:
model_paths = []
k = 5
for i in range(k):
    folder_path = f'weights/'
    
    # Liste tous les fichiers dans le dossier
    all_files = os.listdir(folder_path)
    
    # Trouve le fichier qui termine par '__best.pt'
    best_file = next((f for f in all_files), None)
    
    if best_file:
        full_path = os.path.join(folder_path, best_file)
        model_paths.append(full_path)

In [ ]:
model_paths

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'Device: {device}')

In [ ]:
def extract_number_from_string(s):
    return int(re.search(r'\d+', s).group())

In [ ]:
df = pd.read_csv(mapping_path)

# Verify that video names are unique
assert(len(df.index) == len(df.directory.unique()))

In [ ]:
threshold = 0.5

In [ ]:
class Prediction:
    def __init__(self, frame, proba, label=None):
        self.frame:int = frame
        self.proba:float = proba
        self.label:int = label

    def __str__(self):
        return f"Prediction(frame={self.frame}, proba={self.proba:.4f}, label={self.label})"

In [ ]:
class PredictionList():
    def __init__(self, predictions, video_id, k):
        self.predictions: List[Prediction] = predictions
        self.video_id: str = video_id
        self.k:int = k
    
    def __str__(self):
        return f"PredictionList(video_id={self.video_id}, k={self.k}, predictions={self.predictions})"

In [ ]:
class ListOfPredictionList(list):
    def append(self, item: Type[PredictionList]):
        if not isinstance(item, PredictionList):
            raise ValueError("Item must be of type PredictionList")
        super().append(item)

In [ ]:
def run_inference(video_id: str, current_vid_dir: Path) -> ListOfPredictionList:
    result = ListOfPredictionList()

    # For each model.pt
    for i, model_path in enumerate(model_paths):
        preds = PredictionList(k=i, video_id=video_id, predictions=[])
        model = SimpleCNN().to(device)

        checkpoint = torch.load(model_path)
        model.load_state_dict(checkpoint)  
        
        model.eval()
        model.to(device)

        transform = transforms.Compose([
            transforms.Resize((224, 224)), 
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
        ])

        print(f'Start inference for k={i}')
        
        # Loop through all images
        for img_name in tqdm(os.listdir(current_vid_dir), desc='infering...'):
            if img_name.endswith('.png'):
                # Load image
                img_path = os.path.join(current_vid_dir, img_name)
                img = Image.open(img_path)
            
                # Run inference
                with torch.inference_mode():
                    transformed_img = transform(img).unsqueeze(dim=0)  # Add a batch dimension
                    pred = model(transformed_img.to(device))

                    probabilities = torch.sigmoid(pred)
                    predicted_label = (probabilities > threshold).int()

                frame_number = extract_number_from_string(img_name.split('.')[0])
                
                # Save Inference Result
                pred = Prediction(proba=probabilities.item(), frame=frame_number, label=predicted_label)
                preds.predictions.append(pred)
        
        # Add predictions for this 'k'
        result.append(preds)
        print(f'Finished inference for k={i}')

    return result

In [ ]:
def compute_avg_preds(video_id: str, predictions: ListOfPredictionList):
    result = None
    # Store all probas for each frame_number
    frame_proba_dict = {}

    # Iterate in PredictionList
    for pred_list in predictions:
        # Iterate in  Prediction of PredictionList
        for pred in pred_list.predictions:
            frame = pred.frame
            proba = pred.proba

            # Add proba
            if frame not in frame_proba_dict:
                frame_proba_dict[frame] = []
            
            frame_proba_dict[frame].append(proba)

    # Compute the average
    result = PredictionList(k=None, video_id=video_id, predictions=[])
    for frame, probas in frame_proba_dict.items():
        avg_prob = np.mean(probas)
        avg_label = 0 if avg_prob < threshold else 1
        pred = Prediction(frame=frame, proba=avg_prob, label=avg_label)
        # Add to PredictionList
        result.predictions.append(pred)

    # Sort results by growing frame number
    result.predictions = sorted(result.predictions, key=lambda x: x.frame)

    return result

In [ ]:
def append_to_json(avg_preds: PredictionList, file_name: str = 'predictions.json'):
    # Initialize an empty dictionary
    video_dict = {}
    
    # Create a list to hold frame and label information
    frame_label_list = []
    
    # Loop through each Prediction object in avg_preds
    for prediction in avg_preds.predictions:
        # Create a dictionary for each frame
        frame_dict = {}
        frame_dict["frame"] = prediction.frame
        frame_dict["label"] = prediction.label 
        
        # Append to the list
        frame_label_list.append(frame_dict)
    
    # Add the list to the video dictionary with the key as video_id
    video_dict[avg_preds.video_id] = frame_label_list
    
    # Load existing data from the JSON file if it exists
    try:
        with open(file_name, 'r') as json_file:
            existing_data = json.load(json_file)
    except FileNotFoundError:
        existing_data = {}
    
    # Update the existing data with new video_dict
    existing_data.update(video_dict)
    
    # Save the updated dictionary back to the JSON file
    with open(file_name, 'w') as json_file:
        print('Saving to json...')
        json.dump(existing_data, json_file, indent=4)

In [ ]:
def collect_data(checkpoint):
    for index, row in df.iterrows():
        video_id = row['id']
        id_int = int(video_id[3:])

        print(f'Current id: {id_int}')

        if id_int < checkpoint:
            video_name = row['directory']
            current_vid = os.path.join(video_dir, video_name)
            
            # Get predictions for each model
            predictions: ListOfPredictionList = run_inference(video_id, current_vid)
            # Compute the probability of the average model, and get labels from it
            avg_preds = compute_avg_preds(video_id=video_id, predictions=predictions)
            # Add output to json
            append_to_json(avg_preds, 'predictions.json')

In [ ]:
collect_data(checkpoint=108)